In [50]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
path_file = r'C:\Workspace\DataAnalystics\Aux_Emergencial\csv_aux_emergencial\202104_AuxilioEmergencial.csv'

In [4]:
df = spark.read.csv(path_file, encoding='ISO-8859-1', sep='";"', header=True)

In [5]:
df = df.toDF('ANO_MES', 'UF', 'COD_MUNICIPIO','NOME_MUNICIPIO','NIS_BENEF','CPF_BENEF','NOME_BENEF','NIS_RESP','CPF_RESP','NOME_RESP','ENQUADRAMENTO','PARCELA','OBS','VALOR')

In [6]:
df2 = df.withColumn('ANO_MES', regexp_replace('ANO_MES', '"', ''))

In [7]:
df3 = df2.withColumn('VALOR', regexp_replace('VALOR', '"', ''))

In [8]:
df4 = df3.withColumn('VALOR', regexp_replace('VALOR', ',', '.'))

In [9]:
df5 = df4.withColumn('VALOR', col('VALOR').cast('float'))

In [24]:
df_final = df5.filter("COD_MUNICIPIO IS NOT NULL").groupBy(['ANO_MES','COD_MUNICIPIO','ENQUADRAMENTO','PARCELA']).sum('VALOR')

In [26]:
df_final_order = df_final.orderBy(['COD_MUNICIPIO','ENQUADRAMENTO','PARCELA'])

In [46]:
ano_mes = df_final_order.select('ANO_MES').limit(1).collect()

In [47]:
destino = r'C:\Workspace\DataAnalystics\Aux_Emergencial\csv_aux_emergencial'
path_destino = destino+'\\'+ano_mes[0][0]

In [51]:
try:
    os.mkdir(path_destino)
except FileExistsError:
    pass

In [57]:
file_csv = path_destino+'\\'+'AuxilioEmergencial.csv'
df_final_order.toPandas().to_csv(file_csv)

In [59]:
carga_municipios = df5.select('COD_MUNICIPIO', 'NOME_MUNICIPIO', 'UF').distinct().filter("COD_MUNICIPIO IS NOT NULL")

In [60]:
csv_municipio = path_destino+'\\'+'municipios.csv'
carga_municipios.toPandas().to_csv(csv_municipio)

In [3]:
path_download = 'C:\\Users\\antoliverjr\\Downloads'
path_historico = 'C:\\Workspace\\DataAnalystics\\Aux_Emergencial\\historico_zip'
path_files = 'C:\\Workspace\\DataAnalystics\\Aux_Emergencial\\csv_aux_emergencial'
url_download = 'https://www.portaldatransparencia.gov.br/download-de-dados/auxilio-emergencial/'

try: os.mkdir(path_files)
except FileExistsError: pass

try: os.mkdir(path_historico)
except FileExistsError: pass

file_name = '202104_AuxilioEmergencial.zip'
path_file_origin = os.path.join(path_download, file_name)
path_file_job = os.path.join(path_historico, file_name)

print(file_name)
print(path_historico)

for files in os.listdir(path_historico):
    print(files)

202104_AuxilioEmergencial.zip
C:\Workspace\DataAnalystics\Aux_Emergencial\historico_zip
